In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

2025-04-03 12:26:42.071800: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743683202.121446    2005 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743683202.134914    2005 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743683202.236858    2005 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743683202.236975    2005 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743683202.236977    2005 computation_placer.cc:177] computation placer alr

In [2]:
train = pd.read_csv("../DATA/train.csv")

/tmp/ipykernel_2005/2678278497.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("../DATA/train.csv")


In [3]:
train = train.dropna()

In [4]:
train['Protocol'].unique()

array([6, 17, 0, '6', '17', '0', 'Protocol'], dtype=object)

In [5]:
train['Protocol'] = pd.to_numeric(train['Protocol'], errors='coerce') 
train = train.dropna(subset=['Protocol']) 
train['Protocol'] = train['Protocol'].astype(int) 

In [6]:
train['Protocol'].unique()

array([ 6, 17,  0])

In [7]:
train.columns

Index(['Dst Port', 'Protocol', 'Flow Duration', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Fwd PSH Flags', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'Down/Up Ratio', 'Label'],
      dtype='object')

In [8]:
columns = list(train.columns) 
columns = [col for col in columns if col not in ['Fwd PSH Flags', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Protocol', 'Label']]

In [9]:
x = train.drop(columns=['Label'])
y = train['Label']

In [10]:
x[columns] = x[columns].apply(pd.to_numeric, errors='coerce')


In [11]:
f=['Fwd PSH Flags', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt']
x[f] = x[f].apply(pd.to_numeric, errors='coerce')
x = x.fillna(0)

In [12]:
sc = MinMaxScaler()
x[columns] = sc.fit_transform(x[columns])  
joblib.dump(sc, 'scaler.pkl')  

['scaler.pkl']

In [13]:
class FastRBM(tf.keras.Model):
    def __init__(self, inp_dim, hid_dim, lr=0.01, momentum=0.9, use_fp16=False):
        super(FastRBM, self).__init__()
        self.hid_dim = hid_dim
        self.inp_dim = inp_dim
        self.momentum = momentum
        self.use_fp16 = use_fp16
        self.mixed_precision_dtype = tf.float16 if use_fp16 else tf.float32
        
        initializer = tf.keras.initializers.GlorotUniform()
        self.W = tf.Variable(initializer([inp_dim, hid_dim]), dtype=tf.float32)
        self.bh = tf.Variable(tf.zeros([hid_dim]), dtype=tf.float32)
        self.bv = tf.Variable(tf.zeros([inp_dim]), dtype=tf.float32)
        
        self.W_velocity = tf.Variable(tf.zeros([inp_dim, hid_dim]), dtype=tf.float32, trainable=False)
        self.bh_velocity = tf.Variable(tf.zeros([hid_dim]), dtype=tf.float32, trainable=False)
        self.bv_velocity = tf.Variable(tf.zeros([inp_dim]), dtype=tf.float32, trainable=False)
        
        self.opt = tf.keras.optimizers.Adam(lr)
    
    @tf.function
    def sample_hid(self, v):
        v_cast = tf.cast(v, self.mixed_precision_dtype)
        logits = tf.matmul(v_cast, tf.cast(self.W, self.mixed_precision_dtype)) + tf.cast(self.bh, self.mixed_precision_dtype)
        h_prob = tf.nn.sigmoid(logits)
        random_tensor = tf.random.uniform(tf.shape(h_prob), dtype=self.mixed_precision_dtype)
        h_sample = tf.cast(h_prob > random_tensor, self.mixed_precision_dtype)
        return tf.cast(h_prob, tf.float32), tf.cast(h_sample, tf.float32)
    
    @tf.function
    def sample_vis(self, h):
        h_cast = tf.cast(h, self.mixed_precision_dtype)
        logits = tf.matmul(h_cast, tf.transpose(tf.cast(self.W, self.mixed_precision_dtype))) + tf.cast(self.bv, self.mixed_precision_dtype)
        v_prob = tf.nn.sigmoid(logits)
        random_tensor = tf.random.uniform(tf.shape(v_prob), dtype=self.mixed_precision_dtype)
        v_sample = tf.cast(v_prob > random_tensor, self.mixed_precision_dtype)
        return tf.cast(v_prob, tf.float32), tf.cast(v_sample, tf.float32)
    
    @tf.function
    def contrastive_divergence_step(self, v0, k=1):
    # Ensure consistent dtype for input
        v0 = tf.cast(v0, tf.float32)
    
        h0_prob, h0_sample = self.sample_hid(v0)
        vk_sample = v0  # This will now be float32
        hk_sample = h0_sample
    
        for _ in tf.range(k):
            vk_prob, vk_sample = self.sample_vis(hk_sample)
            hk_prob, hk_sample = self.sample_hid(vk_sample)
        
        batch_size = tf.cast(tf.shape(v0)[0], tf.float32)
        
        pos_grad = tf.matmul(tf.transpose(v0), h0_prob)
        neg_grad = tf.matmul(tf.transpose(vk_sample), hk_prob)
        
        w_grad = (pos_grad - neg_grad) / batch_size
        bh_grad = tf.reduce_mean(h0_prob - hk_prob, axis=0)
        bv_grad = tf.reduce_mean(v0 - vk_sample, axis=0)
        
        lr = self.opt.learning_rate
        self.W_velocity.assign(self.momentum * self.W_velocity + lr * w_grad)
        self.bh_velocity.assign(self.momentum * self.bh_velocity + lr * bh_grad)
        self.bv_velocity.assign(self.momentum * self.bv_velocity + lr * bv_grad)
        
        self.W.assign_add(self.W_velocity)
        self.bh.assign_add(self.bh_velocity)
        self.bv.assign_add(self.bv_velocity)
    
        error = tf.reduce_mean(tf.square(v0 - vk_prob))
        return error    
    def train(self, data, epochs=10, batch_size=256, k=1, progress_bar=True):
        dataset = (tf.data.Dataset.from_tensor_slices(data)
                   .cache()
                   .shuffle(10000)
                   .batch(batch_size, drop_remainder=True)
                   .prefetch(tf.data.AUTOTUNE))
        
        num_batches = len(list(dataset.as_numpy_iterator()))
        
        for epoch in range(epochs):
            if progress_bar:
                batch_iter = tqdm(dataset, total=num_batches, desc=f"Epoch {epoch+1}/{epochs}")
            else:
                batch_iter = dataset
                
            total_error = 0.0
            
            for batch in batch_iter:
                error = self.contrastive_divergence_step(batch, k=k)
                total_error += error
            
            avg_error = total_error / num_batches
            print(f"Epoch {epoch+1}/{epochs}, Avg Error: {avg_error:.6f}")
    
    @tf.function
    def transform(self, data):
        v_cast = tf.cast(data, self.mixed_precision_dtype)
        logits = tf.matmul(v_cast, tf.cast(self.W, self.mixed_precision_dtype)) + tf.cast(self.bh, self.mixed_precision_dtype)
        h_prob = tf.nn.sigmoid(logits)
        return tf.cast(h_prob, tf.float32)

In [14]:
class FastDBN:
    def __init__(self, layer_dims, learning_rate=0.01, momentum=0.9, use_fp16=False):
        self.layer_dims = layer_dims
        self.rbm_layers = []
        self.use_fp16 = use_fp16
        
        for i in range(len(layer_dims) - 1):
            self.rbm_layers.append(
                FastRBM(layer_dims[i], layer_dims[i + 1], 
                       lr=learning_rate, momentum=momentum, use_fp16=use_fp16)
            )
    
    def pretrain(self, data, epochs_per_layer=10, batch_size=256, k=1, progress_bar=True):
        print("Starting high-speed greedy layer-wise pretraining...")
        
        if isinstance(data, np.ndarray):
            data = tf.convert_to_tensor(data, dtype=tf.float32)
        else:
            data = tf.cast(data, tf.float32)
        
        input_data = data
        
        for i, rbm in enumerate(self.rbm_layers):
            print(f"\nPretraining layer {i+1}/{len(self.rbm_layers)}...")
            layer_start_time = tf.timestamp()
            
            rbm.train(input_data, epochs=epochs_per_layer, 
                     batch_size=batch_size, k=k, progress_bar=progress_bar)
            
            if tf.shape(input_data)[0] > 10000:
                chunk_size = 10000
                num_chunks = tf.shape(input_data)[0] // chunk_size + 1
                
                transformed_chunks = []
                for j in range(num_chunks):
                    start_idx = j * chunk_size
                    end_idx = tf.minimum(start_idx + chunk_size, tf.shape(input_data)[0])
                    if end_idx > start_idx:
                        chunk = input_data[start_idx:end_idx]
                        transformed_chunks.append(rbm.transform(chunk))
                
                input_data = tf.concat(transformed_chunks, axis=0)
            else:
                input_data = rbm.transform(input_data)
            
            elapsed = tf.timestamp() - layer_start_time
            print(f"Layer {i+1} training completed in {elapsed:.2f} seconds")
    
    def transform(self, data, in_chunks=True):
        if isinstance(data, np.ndarray):
            data = tf.convert_to_tensor(data, dtype=tf.float32)
            
        input_data = data
        
        if in_chunks and tf.shape(input_data)[0] > 10000:
            chunk_size = 10000
            
            for rbm in self.rbm_layers:
                transformed_chunks = []
                
                num_chunks = tf.shape(input_data)[0] // chunk_size + 1
                for j in range(num_chunks):
                    start_idx = j * chunk_size
                    end_idx = tf.minimum(start_idx + chunk_size, tf.shape(input_data)[0])
                    if end_idx > start_idx:
                        chunk = input_data[start_idx:end_idx]
                        transformed_chunks.append(rbm.transform(chunk))
                
                input_data = tf.concat(transformed_chunks, axis=0)
        else:
            for rbm in self.rbm_layers:
                input_data = rbm.transform(input_data)
                
        return input_data


In [15]:
input_dim = x.shape[1]
layer_dims = [input_dim, 128, 64, 32]
    
    # Create and pretrain the DBN
dbn = FastDBN(layer_dims, learning_rate=0.01, momentum=0.9, use_fp16=True)
dbn.pretrain(x, epochs_per_layer=5, batch_size=1024, k=1)

I0000 00:00:1743683260.100709    2005 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20974 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:35:00.0, compute capability: 8.9


Starting high-speed greedy layer-wise pretraining...


2025-04-03 12:27:41.805898: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1655760152 exceeds 10% of free system memory.
2025-04-03 12:27:42.842868: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 827880076 exceeds 10% of free system memory.



Pretraining layer 1/3...


2025-04-03 12:27:43.257604: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 827880076 exceeds 10% of free system memory.
2025-04-03 12:28:01.453828: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Epoch 1/5: 100%|██████████| 11889/11889 [00:16<00:00, 706.49it/s]


...

Epoch 3/5, Avg Error: 0.000000


Epoch 4/5: 100%|██████████| 11889/11889 [00:15<00:00, 746.65it/s]


Epoch 4/5, Avg Error: 0.000000


Epoch 5/5: 100%|██████████| 11889/11889 [00:15<00:00, 748.16it/s]


Epoch 5/5, Avg Error: 0.000000
Layer 3 training completed in 102.09 seconds


In [16]:
# Save the trained DBN weights using manual saving
for i, rbm in enumerate(dbn.rbm_layers):
    weights_dict = {
        'W': rbm.W.numpy(),
        'bh': rbm.bh.numpy(),
        'bv': rbm.bv.numpy()
    }
    np.savez(f"rbm_layer_{i+1}_weights.npz", **weights_dict)

In [17]:
transformed_x = dbn.transform(x.values)
np.save("transformed_features.npy", transformed_x)